Setup (Install, mount drive, pull code)

In [ ]:
# --- Framework flags: tell Transformers to avoid TensorFlow/Flax ---
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- Install deps (pin requests so Drive auth stays happy) ---
!pip install -U "requests==2.32.4" datasets langchain langchain-community langchain-chroma chromadb sentence-transformers tqdm langchain-huggingface

# --- Hard restart the runtime so the env flags are in effect for all later imports ---
import os, sys
os.kill(os.getpid(), 9)


In [ ]:
# Mount drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
# Verify CUDA is usable
import torch, requests
print("requests:", requests.__version__)
print("torch:", torch.__version__, "cuda_available:", torch.cuda.is_available())
!nvidia-smi -L || true
!nvidia-smi || true

In [ ]:
# Shows a python process + rising memory while embeddings are computed
!nvidia-smi -l 5


Build Script

In [ ]:
import os, time, threading, subprocess
from typing import List
from datasets import load_dataset
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import chromadb  # for PersistentClient

# ---------------- Config ----------------
PERSIST_DIR    = "/content/drive/MyDrive/colab_chroma_10k"   # lives in Drive
COLLECTION     = "edgar_10k"
INDEX_SECTIONS = {"item_1", "item_1a", "item_7"}       # start with key sections
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

TOTAL_ROWS = 6282   # train split size; used for ETA projection

os.makedirs(PERSIST_DIR, exist_ok=True)

# ---------------- Embeddings on GPU (PyTorch) ----------------
emb = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"batch_size": 512, "normalize_embeddings": True},
)

# ---------------- Vector store (auto-persist via PersistentClient) ----------------
client = chromadb.PersistentClient(path=PERSIST_DIR)
vectordb = Chroma(collection_name=COLLECTION, embedding_function=emb, client=client)

# ---------------- Helpers ----------------
def row_to_docs(row) -> List[Document]:
    cik = str(row["cik"])
    date_str = str(row["date"])[:10]
    year = int(date_str[:4])
    doc_id = f"{cik}-{date_str}"
    docs: List[Document] = []
    for k, v in row.items():
        if not isinstance(v, str):
            continue
        k_lower = k.lower()
        if not (k_lower.startswith("item_") and v.strip()):
            continue
        if INDEX_SECTIONS and k_lower not in INDEX_SECTIONS:
            continue
        for idx, chunk in enumerate(splitter.split_text(v)):
            docs.append(Document(
                page_content=chunk,
                metadata={
                    "doc_id": doc_id, "cik": cik, "company": row["company"],
                    "date": date_str, "year": year, "item": k_lower,
                    "chunk_idx": idx, "row_idx": int(row.get("__index_level_0__", -1)),
                }
            ))
    return docs

def doc_id_for(d: Document):
    m = d.metadata
    return f"{m['doc_id']}|{m['item']}|{m['chunk_idx']}"

# ---------------- Heartbeat + Rate/ETA ----------------
print("✅ Starting streaming index…", flush=True)

stop_heartbeat = False
progress = {"rows": 0, "docs": 0, "last_batch": 0}
start = time.time()
ema_rate = None
ALPHA = 0.25

def gpu_stats():
    try:
        out = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=utilization.gpu,memory.used", "--format=csv,noheader,nounits"]
        ).decode().strip()
        util, mem = out.split(',')
        return int(util), int(mem)   # percent, MiB
    except Exception:
        return None, None

def rate_eta():
    global ema_rate
    elapsed = max(time.time() - start, 1e-6)
    inst_rate = progress["docs"] / elapsed
    ema_rate = inst_rate if ema_rate is None else (ALPHA*inst_rate + (1-ALPHA)*ema_rate)
    est_total = progress["docs"] if progress["rows"] == 0 else int((progress["docs"]/progress["rows"]) * TOTAL_ROWS)
    remaining = max(est_total - progress["docs"], 0)
    eta_sec = remaining / ema_rate if ema_rate and ema_rate > 0 else float("inf")
    return ema_rate, eta_sec, est_total

def heartbeat():
    while not stop_heartbeat:
        util, mem = gpu_stats()
        r, eta, est_total = rate_eta()
        eta_min = "?" if eta == float("inf") else f"{eta/60:.1f}m"
        try:
            size = subprocess.check_output(["du","-sh",PERSIST_DIR]).decode().split()[0]
        except Exception:
            size = "?"
        print(
            f"[{time.strftime('%H:%M:%S')}] rows={progress['rows']} docs={progress['docs']} "
            f"last={progress['last_batch']} db={size} "
            f"gpu={util if util is not None else '-'}% mem={mem if mem is not None else '-'}MiB "
            f"rate={0 if r is None else r:.1f}/s ETA~{eta_min} (est_total≈{est_total:,})",
            flush=True
        )
        time.sleep(60)

threading.Thread(target=heartbeat, daemon=True).start()

# ---------------------------------------- (1) Knowledge Base ----------------------------------------
KNOWLEDGE_BASE = "jlohding/sp500-edgar-10k"

# ---------------------------------------- (2) Load ----------------------------------------
dataset = load_dataset(KNOWLEDGE_BASE, split="train", streaming=True)

# ---------------------------------------- (3) Preprocess & Chunk ----------------------------------------
BATCH = 500
LOG_EVERY = 10  # formerly PERSIST_EVERY
since_log = 0
buf_docs, buf_ids = [], []
try:
    with tqdm(unit="vec", mininterval=2, desc="Indexing") as pbar:
        for row in dataset:
            progress["rows"] += 1

            # ---------------------------------------- (4) Embed & Index in Vector Store ----------------------------------------
            for d in row_to_docs(row):
                buf_docs.append(d); buf_ids.append(doc_id_for(d))
                if len(buf_docs) >= BATCH:
                    vectordb.add_documents(buf_docs, ids=buf_ids)  # writes to disk
                    progress["docs"] += len(buf_docs)
                    progress["last_batch"] = len(buf_docs)
                    pbar.update(len(buf_docs))
                    buf_docs.clear(); buf_ids.clear()
except KeyboardInterrupt:
    if buf_docs:
        vectordb.add_documents(buf_docs, ids=buf_ids)  # flush remainder
        progress["docs"] += len(buf_docs)
        print(f"Interrupted — flushed {len(buf_docs)} docs.", flush=True)
finally:
    stop_heartbeat = True
r, eta, est = rate_eta()
print(f"✅ Done. Chroma DB at: {PERSIST_DIR}")
print(f"Final rate≈{r:.1f} docs/s • total_docs={progress['docs']:,}")


In [ ]:
import os, time, subprocess, shutil, chromadb

SRC = "/content/drive/MyDrive/colab_chroma_10k"
STAMP = time.strftime("%Y%m%d_%H%M%S")
SNAP = f"/content/drive/MyDrive/colab_chroma_10k_snap_{STAMP}"

# 1) Touch the DB once to let SQLite clean any journal
client = chromadb.PersistentClient(path=SRC)
print("Collections:", [c.name for c in client.list_collections()])
del client  # close handle

# 2) Snapshot the folder (so you zip a stable copy)
os.makedirs(SNAP, exist_ok=True)
subprocess.run(["rsync","-a","--delete", f"{SRC}/", f"{SNAP}/"], check=True)
print("Snapshot:", SNAP)


In [ ]:
import shutil
ZIP_BASE = f"/content/drive/MyDrive/edgar_10k_chroma_{STAMP}"
shutil.make_archive(ZIP_BASE, "zip", SNAP)
print("Created:", ZIP_BASE + ".zip")


In [ ]:
from google.colab import files
files.download(ZIP_BASE + ".zip")
